## Importing Libraries

import numpy as np

import cv2
import os, sys
import time
import operator
import tensorflow as tf
from string import ascii_uppercase

import tkinter as tk
from PIL import Image, ImageTk

import tkinter as tk
from ttkthemes import ThemedTk
import enchant
from ttkthemes import ThemedTk, ThemedStyle
from keras.models import model_from_json


#Application :

class Application:

    def __init__(self):

        
        self.vs = cv2.VideoCapture(0)
        self.current_image = None
        self.current_image2 = None
        self.json_file = open("model_new.json", "r")
        self.model_json = self.json_file.read()
        self.json_file.close()
        self.loaded_model = model_from_json(self.model_json)
        self.loaded_model.load_weights("model_new.h5")

        self.json_file_dru = open("model_new_dru.json" , "r")
        self.model_json_dru = self.json_file_dru.read()
        self.json_file_dru.close()
        self.loaded_model_dru = model_from_json(self.model_json_dru)
        self.loaded_model_dru.load_weights("model_new_dru.h5")
        
        
        self.json_file_tkdi = open("model_new_tkdi.json" , "r")
        self.model_json_tkdi = self.json_file_tkdi.read()
        self.json_file_tkdi.close()
        self.loaded_model_tkdi = model_from_json(self.model_json_tkdi)
        self.loaded_model_tkdi.load_weights("model_new_tkdi.h5")
        
        
        self.json_file_smn = open("model_new_smn.json" , "r")
        self.model_json_smn = self.json_file_smn.read()
        self.json_file_smn.close()
        self.loaded_model_smn = model_from_json(self.model_json_smn)
        self.loaded_model_smn.load_weights("model_new_smn.h5")
        
        self.json_file_fbi = open("model_new_fbi.json" , "r")
        self.model_json_fbi = self.json_file_fbi.read()
        self.json_file_fbi.close()
        self.loaded_model_fbi = model_from_json(self.model_json_fbi)
        self.loaded_model_fbi.load_weights("model_new_fbi.h5")

        self.ct = {}
        self.ct['blank'] = 0
        self.blank_flag = 0

        for i in ascii_uppercase:
            self.ct[i] = 0
        
        print("Loaded model from disk")

        self.root = ThemedTk(theme="equilux")  # Choose the theme you prefer
        self.root.get_themes()  # Get available themes

        self.root.set_theme("equilux")  # Set the theme (you can change this to a different theme)

        self.root.title("ASL CONVERTER")
        self.root.protocol('WM_DELETE_WINDOW', self.destructor)
        self.root.geometry("900x900")
        self.root.configure(bg="#72b1b7")
        # Main Title (Header)
        self.T = tk.Label(self.root, text="Sign Language To Text Conversion", font=("Arial", 24, "bold"), background="#A4C7E7", foreground="white")
        self.T.pack(fill=tk.X)

        # Image Panels (Body)
        self.panel = tk.Label(self.root, background="#5a99d1")
        self.panel.place(x=100, y=50, width=580, height=580)
        
        self.panel2 = tk.Label(self.root, background="#5a99d1")
        self.panel2.place(x=400, y=65, width=275, height=275)

        # Labels for Character, Word, Sentence (Body)
        label_font = ("Arial", 16, "bold")

        self.T1 = tk.Label(self.root, text="Letter :", font=label_font, background="#72b1b7", foreground="black")
        self.T1.place(x=10, y=670)

        self.T2 = tk.Label(self.root, text="Word :", font=label_font, background="#72b1b7", foreground="black")
        self.T2.place(x=10, y=720)

        self.T3 = tk.Label(self.root, text="Sentence:", font=label_font, background="#72b1b7", foreground="black")
        self.T3.place(x=10, y=770)

        # Display for Current Symbol and Word (Body)
        display_font = ("Arial", 16)

        self.panel3 = tk.Label(self.root, background="#72b1b7", font=display_font)
        self.panel3.place(x=140, y=670)

        self.panel4 = tk.Label(self.root, background="#72b1b7", font=display_font)
        self.panel4.place(x=140, y=720)

        self.panel5 = tk.Label(self.root, background="#72b1b7", font=display_font)
        self.panel5.place(x=140, y=770)

        self.select_letter_button = tk.Button(
        self.root, text="Select Letter", font=("Arial", 14),
        command=self.select_letter)
        self.select_letter_button.place(x=400, y=700)  # Adjusted y-coordinate
    
    # Delete Button
        self.delete_button = tk.Button(
        self.root, text="Delete", font=("Arial", 14),
        command=self.delete_letter)
        self.delete_button.place(x=550, y=700)


        

        


        self.str = ""
        self.word = " "
        self.current_symbol = "Empty"
        self.photo = "Empty"
        self.video_loop()


    def video_loop(self):
        ok, frame = self.vs.read()

        if ok:
            cv2image = cv2.flip(frame, 1)

            x1 = int(0.5 * frame.shape[1])
            y1 = 10
            x2 = frame.shape[1] - 10
            y2 = int(0.5 * frame.shape[1])

            cv2.rectangle(frame, (x1 - 1, y1 - 1), (x2 + 1, y2 + 1), (255, 0, 0) ,1)
            cv2image = cv2.cvtColor(cv2image, cv2.COLOR_BGR2RGBA)

            self.current_image = Image.fromarray(cv2image)
            imgtk = ImageTk.PhotoImage(image = self.current_image)

            self.panel.imgtk = imgtk
            self.panel.config(image = imgtk)

            cv2image = cv2image[y1 : y2, x1 : x2]

            gray = cv2.cvtColor(cv2image, cv2.COLOR_BGR2GRAY)

            blur = cv2.GaussianBlur(gray, (5, 5), 2)

            th3 = cv2.adaptiveThreshold(blur, 255 ,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

            ret, res = cv2.threshold(th3, 70, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
            
            res=cv2.cvtColor(res,cv2.COLOR_GRAY2RGB)
            
            self.predict(res)

            self.current_image2 = Image.fromarray(res)

            imgtk = ImageTk.PhotoImage(image = self.current_image2)

            self.panel2.imgtk = imgtk
            self.panel2.config(image = imgtk)

            self.panel3.config(text = self.current_symbol, font = ("Courier", 30))

            self.panel4.config(text = self.word, font = ("Courier", 30))

            self.panel5.config(text = self.str,font = ("Courier", 30))

           

              


        self.root.after(5, self.video_loop)

    def predict(self, test_image):

        test_image_resized = cv2.resize(test_image, (128, 128))
        
        test_image_normalized = tf.keras.applications.resnet50.preprocess_input(test_image_resized)
        
        test_image_shaped=np.expand_dims(test_image_normalized,axis=0)

        result = self.loaded_model.predict(test_image_shaped)


        result_dru = self.loaded_model_dru.predict(test_image_shaped)

        result_tkdi = self.loaded_model_tkdi.predict(test_image_shaped)

        result_smn = self.loaded_model_smn.predict(test_image_shaped)
        
        result_fbi = self.loaded_model_fbi.predict(test_image_shaped)

        prediction = {}

        prediction['blank'] = result[0][0]

        inde = 1

        for i in ascii_uppercase:

            prediction[i] = result[0][inde]

            inde += 1

        #LAYER 1

        prediction = sorted(prediction.items(), key = operator.itemgetter(1), reverse = True)

        self.current_symbol = prediction[0][0]


#         #LAYER 2

        if(self.current_symbol == 'D' or self.current_symbol == 'R' or self.current_symbol == 'U'):

        	prediction = {}

        	prediction['D'] = result_dru[0][0]
        	prediction['R'] = result_dru[0][1]
        	prediction['U'] = result_dru[0][2]

        	prediction = sorted(prediction.items(), key = operator.itemgetter(1), reverse = True)

        	self.current_symbol = prediction[0][0]
        
        
        
        if(self.current_symbol == 'F' or self.current_symbol == 'B' or self.current_symbol == 'I'):

        	prediction = {}

        	prediction['F'] = result_fbi[0][0]
        	prediction['B'] = result_fbi[0][1]
        	prediction['I'] = result_fbi[0][2]

        	prediction = sorted(prediction.items(), key = operator.itemgetter(1), reverse = True)

        	self.current_symbol = prediction[0][0]
        
        
        
        

        if(self.current_symbol == 'D' or self.current_symbol == 'I' or self.current_symbol == 'K' or self.current_symbol == 'T'):

        	prediction = {}

        	prediction['D'] = result_tkdi[0][0]
        	prediction['I'] = result_tkdi[0][1]
        	prediction['K'] = result_tkdi[0][2]
        	prediction['T'] = result_tkdi[0][3]
            

        	prediction = sorted(prediction.items(), key = operator.itemgetter(1), reverse = True)

        	self.current_symbol = prediction[0][0]

        if(self.current_symbol == 'M' or self.current_symbol == 'N' or self.current_symbol == 'S'):

        	prediction1 = {}

        	prediction1['M'] = result_smn[0][0]
        	prediction1['N'] = result_smn[0][1]
        	prediction1['S'] = result_smn[0][2]

        	prediction1 = sorted(prediction1.items(), key = operator.itemgetter(1), reverse = True)

        	if(prediction1[0][0] == 'S'):

        		self.current_symbol = prediction1[0][0]

        	else:

        		self.current_symbol = prediction[0][0]
        
        if(self.current_symbol == 'blank'):

            for i in ascii_uppercase:
                self.ct[i] = 0

        self.ct[self.current_symbol] += 1
#transition code

        
        if(self.ct[self.current_symbol] > 6):

            for i in ascii_uppercase:
                if i == self.current_symbol:
                    continue

                tmp = self.ct[self.current_symbol] - self.ct[i]

                if tmp < 0:
                    tmp *= -1

                if tmp <= 4:
                    self.ct['blank'] = 0

                    for i in ascii_uppercase:
                        self.ct[i] = 0
                    return

            self.ct['blank'] = 0

            for i in ascii_uppercase:
                self.ct[i] = 0

            if self.current_symbol == 'blank':

                if self.blank_flag == 0:
                    self.blank_flag = 1

                    if len(self.str) > 0:
                        self.str += " "

                    self.str += self.word

                    self.word = ""

            else:

                if(len(self.str) > 1):
                    self.str = ""

                self.blank_flag = 0

                self.word += self.current_symbol

    def select_letter(self):
        if self.current_symbol != "Empty":
            self.word += self.current_symbol
            self.panel4.config(text=self.word, font=("Courier", 30))

    def delete_letter(self):
        if len(self.word) > 0:
            self.word = self.word[:-1]
            self.panel4.config(text=self.word, font=("Courier", 30))

    
            
    def destructor(self):

        print("Closing Application...")

        self.root.destroy()
        self.vs.release()
        cv2.destroyAllWindows()
    
print("Starting Application...")

(Application()).root.mainloop()